In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
order_df=pd.read_csv('/content/drive/My Drive/logistics-shopee-code-league/delivery_orders_march.csv')

In [ ]:
order_df['buyeraddress']=order_df['buyeraddress'].str.lower()
order_df['selleraddress']=order_df['selleraddress'].str.lower()
place=['metro manila','luzon','visayas','mindanao']
for ele in place: 
  order_df['tempIndex']=order_df['buyeraddress'].str.find(ele) 
  order_df['tempIndex1']=order_df['selleraddress'].str.find(ele)
  order_df.loc[order_df['tempIndex']!=-1,'buyeraddress']=ele
  order_df.loc[order_df['tempIndex1']!=-1,'selleraddress']=ele
order_df=order_df.drop(columns=['tempIndex','tempIndex1'])

In [ ]:
order_df['pick_date'] = pd.to_datetime(order_df['pick'],unit = 's').dt.tz_localize("GMT").dt.tz_convert('Singapore').dt.tz_localize(None)
order_df['delivery_date'] = pd.to_datetime(order_df['1st_deliver_attempt'],unit = 's').dt.tz_localize("GMT").dt.tz_convert('Singapore').dt.tz_localize(None)
order_df['delivery2_date'] = pd.to_datetime(order_df['2nd_deliver_attempt'],unit = 's').dt.tz_localize("GMT").dt.tz_convert('Singapore').dt.tz_localize(None)
order_df['pick_day'] = order_df['pick_date'].dt.dayofweek
order_df['delivery_day'] = order_df['delivery_date'].dt.dayofweek
order_df['delivery2_day'] = order_df['delivery2_date'].dt.dayofweek

In [ ]:
order_df['working days']=7
order_df.loc[(order_df['buyeraddress']=='metro manila') & (order_df['selleraddress']=='metro manila'),'working days']=3
order_df.loc[(order_df['buyeraddress']=='luzon') & (order_df['selleraddress']=='luzon'),'working days']=5
order_df.loc[(order_df['buyeraddress']=='metro manila') & (order_df['selleraddress']=='luzon'),'working days']=5
order_df.loc[(order_df['buyeraddress']=='luzon') & (order_df['selleraddress']=='metro manila'),'working days']=5

In [ ]:
order_df['duration']=order_df['delivery_date'].dt.date-order_df['pick_date'].dt.date
order_df['re_duration']=order_df['delivery2_date'].dt.date-order_df['delivery_date'].dt.date
order_df.drop(columns=['pick','1st_deliver_attempt','2nd_deliver_attempt','buyeraddress','selleraddress'],inplace=True)

CPU times: user 19.5 s, sys: 275 ms, total: 19.8 s
Wall time: 19.8 s


In [ ]:
import datetime
order_df['ph1']=datetime.date(2020,3,8)
order_df['ph2']=datetime.date(2020,3,25)
order_df['ph3']=datetime.date(2020,3,30)
order_df['ph4']=datetime.date(2020,3,31)
order_df['ph5']=datetime.date(2020,3,15)
order_df['ph6']=datetime.date(2020,3,22)
order_df['ph7']=datetime.date(2020,3,29)
order_df['ph8']=datetime.date(2020,4,5)
order_df

In [ ]:
order_df['is_late']=0
order_df['duration_bonus']=0
order_df['re_duration_bonus']=0
for cols in order_df.columns[10:18]:
  order_df.loc[(order_df[cols]>order_df['pick_date'].dt.date) & (order_df['delivery_date'].dt.date>order_df[cols]),'duration_bonus']+=1
  order_df.loc[(order_df[cols]>order_df['delivery_date'].dt.date) & (order_df['delivery2_date'].dt.date>order_df[cols]),'re_duration_bonus']+=1
order_df['duration']=order_df['duration'].dt.days-order_df['duration_bonus']
order_df['re_duration']=order_df['re_duration'].dt.days-order_df['re_duration_bonus']

In [ ]:
order_df.loc[(order_df['working days']<order_df['duration']) | (order_df['re_duration']>3), 'is_late']=1
ans=pd.DataFrame(order_df[['orderid','is_late']])
ans.to_csv('/content/drive/My Drive/logistics.csv',index=False)